In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import networkx as nx
import pickle

# Load minnie65 Connectome

In [7]:
connectome_path = "../Data/minnie65_multidigraph.pkl"

with open(connectome_path, "rb") as f:
    G = pickle.load(f)

G

# Setting the node in degree and out degree

In [1]:
len(G.nodes())

NameError: name 'G' is not defined

In [13]:
import networkx as nx

# Add in-degree and out-degree as node attributes
in_degree_dict = dict(G.in_degree())
out_degree_dict = dict(G.out_degree())
nx.set_node_attributes(G, dict(G.in_degree()), "in_degree")
nx.set_node_attributes(G, dict(G.out_degree()), "out_degree")
nx.set_node_attributes(G, dict(G.degree()),"total_degree")

# Look at the features of a certain node

In [14]:
nodes = list(G.nodes())
nodes[:10]

['864691136023767609_0',
 '864691136965808334_0',
 '864691135617737103_0',
 '864691134939970403_0',
 '864691135497605139_0',
 '864691135366598130_0',
 '864691135385264469_0',
 '864691136296739099_0',
 '864691135939472385_0',
 '864691136023727417_0']

In [15]:
n = "864691136965808334_0"
G.nodes[n]

{'segment_id': 864691136965808334,
 'split_index': 0,
 'multiplicity': 1,
 'cell_type_used': 'baylor',
 'cell_type': 'excitatory',
 'nucleus_id': 33321,
 'nuclei_distance': 490.34,
 'n_nuclei_in_radius': 1,
 'n_nuclei_in_bbox': 1,
 'centroid_x': 73773,
 'centroid_y': 167723,
 'centroid_z': 19855,
 'centroid_x_nm': 295092.4139694181,
 'centroid_y_nm': 670893.6067298498,
 'centroid_z_nm': 794203.913531376,
 'centroid_volume': 410.54936253946556,
 'max_soma_n_faces': 24877,
 'max_soma_volume': 639,
 'max_soma_area': 358,
 'syn_density_post_after_proof': 0.4050608177925558,
 'syn_density_head_after_proof': 0.16643588057812936,
 'syn_density_neck_after_proof': 0.0020052515732304744,
 'syn_density_shaft_after_proof': 0.18849364788366457,
 'skeletal_length_processed_syn_after_proof': 498690.54504168424,
 'spine_density_after_proof': 0.25174413039889243,
 'skeletal_length_processed_spine_after_proof': 432979.3104899322,
 'baylor_cell_type_after_proof': 'excitatory',
 'baylor_cell_type_exc_prob

# Generate the optimal centroid for given attribute

In [ ]:
centroid_x_nm': 295092.4139694181,
'centroid_y_nm': 670893.6067298498,
'centroid_z_nm': 794203.913531376,

In [ ]:
"""
Purpose
-------
Want to generate the 
voxel_to_nm_scaling 
"""

# Optimal region finding function

In [19]:
import pandas as pd

def nodes_to_dataframe(G, nodes, attrs=None):
    """
    Create a DataFrame of node attributes for given nodes.
    If attrs=None, include all attributes.
    """
    if type(attrs) == str:
        attrs = [attrs]
    records = []
    for n in nodes:
        node_attrs = G.nodes[n].copy()
        node_attrs["node_id"] = n
        if attrs:
            node_attrs = {k: node_attrs.get(k) for k in ["node_id"] + list(attrs)}
        records.append(node_attrs)
    return pd.DataFrame(records)


In [21]:
best_nodes = ['864691136023767609_0',
 '864691136965808334_0',
 '864691135617737103_0',
 '864691134939970403_0',
 '864691135497605139_0',
 '864691135366598130_0',
 '864691135385264469_0',
 '864691136296739099_0',
 '864691135939472385_0',
 '864691136023727417_0']

attr_to_use = "apical_mesh_volume"

df = nodes_to_dataframe(
    G,
    best_nodes,
    [attr_to_use]
)

df

,node_id,apical_mesh_volume
0,864691136023767609_0,5772
1,864691136965808334_0,172
2,864691135617737103_0,0
3,864691134939970403_0,2033
4,864691135497605139_0,0
5,864691135366598130_0,0
6,864691135385264469_0,64
7,864691136296739099_0,0
8,864691135939472385_0,0
9,864691136023727417_0,54


In [24]:
import numpy as np

def find_best_region(
    G,
    radius,
    attr_to_use,
    objective = "sum",
    shape = "sphere",
    default_attr_value = 0.0,
    center_attribute = "centroid_x_nm",
    tie_break = "first",
    ):

    """
    Find the node-centered region (sphere/cube/circle/square) of given radius
    that maximizes an objective over a node attribute.

    Returns:
      {
        "center_node": node id of best center,
        "centroid": (x,y,z) of center,
        "nodes_in_region": list of nodes in best region,
        "score": best objective value,
        "df": dataframe of nodes and there attributes of interest
      }
    """
    nodes = list(G.nodes())

    centroid_names = [center_attribute.replace("_x",f"_{k}")
                      for k in ['x','y','z']]
    
    X = np.array([G.nodes[n].get(centroid_names[0], np.nan) for n in nodes], dtype=float)
    Y = np.array([G.nodes[n].get(centroid_names[1], np.nan) for n in nodes], dtype=float)
    Z = np.array([G.nodes[n].get(centroid_names[2], np.nan) for n in nodes], dtype=float)

    if np.isnan(X).any() or np.isnan(Y).any() or np.isnan(Z).any():
        raise ValueError("All nodes must have centroid_x_nm, centroid_y_nm, centroid_z_nm.")

    # Attribute values (only used if attr_to_use is not None and objective != "count")
    vals = None
    if objective != "count" and attr_to_use is not None:
        vals = np.array([G.nodes[n].get(attr_to_use, default_attr_value) for n in nodes], dtype=float)

    # Shape config
    shape = shape.lower()
    if shape not in {"sphere", "cube", "circle", "square"}:
        raise ValueError("shape must be one of: 'sphere', 'cube', 'circle', 'square'.")

    use_3d = shape in {"sphere", "cube"}
    use_euclidean = shape in {"sphere", "circle"}  # else Chebyshev (axis-aligned box)

    coords = np.stack([X, Y, Z], axis=1) if use_3d else np.stack([X, Y], axis=1)
    r = float(radius)
    r2 = r * r

    # Objective resolver
    if callable(objective):
        agg_fn = objective  # expects agg_fn(arr) -> float
        needs_attr = (objective != "count")
    else:
        key = str(objective).lower()
        needs_attr = (key != "count")
        if key in ("sum",):
            agg_fn = np.sum
        elif key in ("mean", "avg", "average"):
            def agg_fn(a): return np.mean(a) if a.size else -np.inf
        elif key == "max":
            def agg_fn(a): return np.max(a) if a.size else -np.inf
        elif key == "min":
            def agg_fn(a): return np.min(a) if a.size else np.inf  # note: maximizing min favors larger mins
        elif key == "median":
            def agg_fn(a): return np.median(a) if a.size else -np.inf
        elif key == "count":
            def agg_fn(a): return float(a.size)
        else:
            raise ValueError("objective must be one of: sum, mean/avg, max, min, median, count, or a callable")

    if needs_attr and vals is None:
        # user asked for an attr-based objective but gave no attr_to_use
        raise ValueError("attr_to_use must be provided for objectives other than 'count' or a custom callable that ignores input.")

    best_score = -np.inf
    best_idx = -1
    best_nodes = None
    best_size = -1

    # Try fast cKDTree, else NumPy fallback
    try:
        from scipy.spatial import cKDTree
        tree = cKDTree(coords)
        p = np.inf if not use_euclidean else 2
        neighbor_lists = tree.query_ball_point(coords, r, p=p)

        for i, neigh in enumerate(neighbor_lists):
            if needs_attr:
                a = vals[neigh]
            else:
                # pass dummy array of ones so agg_fn can use .size for "count"
                a = np.ones(len(neigh), dtype=float)
            score = float(agg_fn(a))
            if (score > best_score) or (score == best_score and tie_break == "largest" and len(neigh) > best_size):
                best_score = score
                best_idx = i
                best_nodes = [nodes[j] for j in neigh]
                best_size = len(neigh)

    except Exception:
        # O(N^2) fallback
        for i in range(coords.shape[0]):
            delta = coords - coords[i]
            if use_euclidean:
                mask = (delta * delta).sum(axis=1) <= r2
            else:
                mask = np.max(np.abs(delta), axis=1) <= r
            idxs = np.flatnonzero(mask)

            if needs_attr:
                a = vals[idxs]
            else:
                a = np.ones(idxs.size, dtype=float)

            score = float(agg_fn(a))
            if (score > best_score) or (score == best_score and tie_break == "largest" and idxs.size > best_size):
                best_score = score
                best_idx = i
                best_nodes = [nodes[j] for j in idxs]
                best_size = idxs.size

    center_node = nodes[best_idx]
    centroid = (float(X[best_idx]), float(Y[best_idx]), float(Z[best_idx]))

    

    return {
        "center_node": center_node,
        "centroid": centroid,
        "nodes_in_region": best_nodes,
        "n_nodes": len(best_nodes),
        "score": best_score,
        "attr_df":nodes_to_dataframe(
            G,
            best_nodes,
            [attr_to_use]
        )
    }


    

In [26]:
r2 = find_best_region(
    G, 
    radius=100_000, 
    attr_to_use="axon_skeletal_length", 
    objective="mean", 
    shape="sphere"
)

r2

{'center_node': '864691135348268247_0',
 'centroid': (1255918.5531486792, 687405.4635343975, 980714.5568474651),
 'nodes_in_region': ['864691135700409211_0',
  '864691135122396967_0',
  '864691136194102230_0',
  '864691136296702747_0',
  '864691135385422677_0',
  '864691135516212947_0',
  '864691135571164653_0',
  '864691135479922884_0',
  '864691136582363810_0',
  '864691135064986948_0',
  '864691136031872699_0',
  '864691135974780527_0',
  '864691135385295445_0',
  '864691136378790101_0',
  '864691135382467290_0',
  '864691135544370162_0',
  '864691136020282616_0',
  '864691135975633475_0',
  '864691135738609812_0',
  '864691136811995507_0',
  '864691136965901006_0',
  '864691135081738743_0',
  '864691135718541617_0',
  '864691137197345601_0',
  '864691135560809441_0',
  '864691135697679637_0',
  '864691135181773570_0',
  '864691135809608652_0',
  '864691135617770383_0',
  '864691135012409078_0',
  '864691135104014925_0',
  '864691135945465892_0',
  '864691136536113826_0',
  '8646911